In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hp\\OneDrive\\Desktop\\e2e\\end-to-end-ml\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hp\\OneDrive\\Desktop\\e2e\\end-to-end-ml'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str
    random_state: int

In [7]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.model_trainer.root_dir])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        schema = self.schema.TARGET_COLUMN
        params = self.params.Elastic_Net

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name,
            random_state=params.random_state
        )
        return model_trainer_config

In [9]:
import pandas as pd
import os
from mlproject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_y = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        model = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio,random_state=self.config.random_state)
        model.fit(train_x, train_y)

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))
        

In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-03-07 01:34:46,869: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-07 01:34:46,880: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-07 01:34:46,886: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-07 01:34:46,888: INFO: common: created directory at: artifacts/model_trainer]
[2025-03-07 01:34:46,888: INFO: common: created directory at: artifacts/model_trainer]
